<a href="https://colab.research.google.com/github/AiratGaliev/text_generation_webui/blob/main/text_generation_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Launch the web UI

#@markdown If unsure about the branch, write "main" or leave it blank.

%cd /content
!apt-get -y install -qq aria2
!git clone https://github.com/oobabooga/text-generation-webui.git

# Parameters
model_url = "TheBloke/dolphin-2.2.1-mistral-7B-GPTQ" #@param {type:"string"}
branch = "gptq-4bit-32g-actorder_True" #@param {type:"string"}
command_line_flags = "--use_fast --max_seq_len 10240" #@param {type:"string"}
api = True #@param {type:"boolean"}
model_folder = "dolphin-2.2.1-mistral-7B-GPTQ" #@param {type:"string"}

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

branch = branch.strip()
if branch == "":
  branch = "main"

model_folder = f"/content/text-generation-webui/models/{model_folder}"

if api:
  for param in ['--api','--public-api','--listen']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

# Download the model
files_to_download = [
    "README.md",
    "added_tokens.json",
    "config.json",
    "generation_config.json",
    "model.safetensors",
    "quantize_config.json",
    "special_tokens_map.json",
    "tokenizer.model",
    "tokenizer_config.json",
    "zero_to_fp32.py"
]

model_url += f"/resolve/{branch}/"
for file in files_to_download:
    download_cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url}{file} -d {model_folder} -o {file}"
    print(download_cmd)
    !$download_cmd

# Start the web UI
%cd /content/text-generation-webui
cmd = f"/bin/bash start_linux.sh --share"
if model_folder != "":
    cmd += f" --model {model_folder}"
cmd += f" {command_line_flags}"
print(cmd)
!$cmd